## 붓꽃 품종 분류 
- 목표 : 붓꽃의 3개 품종 분류 하기 
- 데이터셋 : 내장 데이터셋 사용 
- 피쳐 : 품종 1개 
- 학습 : 지도학습 - 분류

## [1] 데이터 준비

In [281]:
# 모듈 로딩
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [282]:
# 내장 데이터셋 로딩 

data = load_iris(as_frame=True) # as_frame, return_X_y 매개변수 존재

In [283]:
# Bunch 인스턴스 -> dict 와 유사한 형태 
data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [284]:
feature_df = data['data']
target_sr = data['target']

In [285]:
feature_df.shape, target_sr.shape

((150, 4), (150,))

In [286]:
feature_df.head(1)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2


In [287]:
target_sr.head(1)

0    0
Name: target, dtype: int32

## [2] 학습을 위한 데이터셋 준비 -> 학습용, 검증용, 테스트용

In [288]:
# 학습용 , 테스트용 분리 

x_train, x_test, y_train, y_test = train_test_split(feature_df, target_sr, stratify=target_sr) 
# 품종이 3가지이므로 비율 신경써야 함 -> stratify 사용 

In [289]:
# 학습용, 검증용 분리 
# 검증 데이터를 매번 변경 -> 모든 데이터에서 적용 가능하도록 

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, stratify=y_train)

In [290]:
print(f'Train DS : {x_train.shape[0]}') # feature는 안 봐도 되므로 [0]만 가져옴  

Train DS : 84


In [291]:
print(f'Train DS : {x_train.shape[0]}    {x_train.shape[0]/feature_df.shape[0]}%')
print(f'Val DS : {x_val.shape[0]}    {x_val.shape[0]/feature_df.shape[0]:.2f}%')
print(f'TEST DS : {x_test.shape[0]}    {x_test.shape[0]/feature_df.shape[0]:.2f}%')

Train DS : 84    0.56%
Val DS : 28    0.19%
TEST DS : 38    0.25%


## [3] 교차검증 분석

In [292]:
# 모듈 로딩 
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

In [293]:
# 모델 인스턴스 생성 
dtc_model = DecisionTreeClassifier()

### [3-1] Kfold 기반 

In [294]:
# 정확도 저장 리스트 

accuracys = []

# Kfold 인스턴스 생성 [기본 = 5]
kfold = KFold()

In [295]:
# k번 만큼 k개 학습 진행 
# -> k등분 후 학습용 데이터셋 인덱스, 검증용 데이터셋 인덱스 

for train_index, val_index in kfold.split(feature_df): # kfold 안에 있는 속성으로 feature_df 를 자동으로 5등분 함 
    
    print(f'train_index : {train_index.tolist()}')

    # x_trian, x_val 데이터셋 설정 

    x_train, y_train = feature_df.iloc[train_index.tolist()], target_sr[train_index.tolist()]
    x_val, y_val = feature_df.iloc[val_index.tolist()], target_sr[val_index.tolist()]


    # 학습 진행 
    dtc_model.fit(x_train, y_train)

    # 평가 
    accuracy = dtc_model.score(x_val, y_val)
    accuracys.append(accuracy)

train_index : [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149]
train_index : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 

In [296]:
# 위의 코드에서 enumerate 사용 

accuracys = []

for idx, (train_index, val_index) in enumerate(kfold.split(feature_df)): 
    # enumerate(kfold.split(feature_df),1) 주면 1부터 시작 
    
    # print(f'train_index : {train_index.tolist()}')

    # x_trian, x_val 데이터셋 설정 

    x_train, y_train = feature_df.iloc[train_index.tolist()], target_sr[train_index.tolist()]
    x_val, y_val = feature_df.iloc[val_index.tolist()], target_sr[val_index.tolist()]


    # 학습 진행 
    dtc_model.fit(x_train, y_train)

    # 평가 -> 분류의 경우 score() 메서드 => 정확도 반환 
    train_acc = dtc_model.score(x_train, y_train)
    val_acc = dtc_model.score(x_val, y_val)

    accuracys.append([train_acc, val_acc])
    print(f'[{idx}번째] Train 정확도 : {train_acc}  Val 정확도 : {val_acc}')

[0번째] Train 정확도 : 1.0  Val 정확도 : 1.0
[1번째] Train 정확도 : 1.0  Val 정확도 : 1.0
[2번째] Train 정확도 : 1.0  Val 정확도 : 0.8333333333333334
[3번째] Train 정확도 : 1.0  Val 정확도 : 0.9333333333333333
[4번째] Train 정확도 : 1.0  Val 정확도 : 0.8


In [297]:
# 평균 계산 
train_mean = sum([value[0] for value in accuracys]) / kfold.n_splits
val_mean = sum([value[1] for value in accuracys]) / kfold.n_splits

print(f'Train 정확도 : {train_mean}, Val 정확도 : {val_mean:.2f}')

Train 정확도 : 1.0, Val 정확도 : 0.91


In [298]:
accuracys

[[1.0, 1.0],
 [1.0, 1.0],
 [1.0, 0.8333333333333334],
 [1.0, 0.9333333333333333],
 [1.0, 0.8]]

### [3-2] StratifiedKFold 
- 정답/레이블/타겟의 비율을 고려해서 데이터 나눔 

In [299]:
accuracys = []

skFold = StratifiedKFold()

for idx, (train_index, val_index) in enumerate(skFold.split(feature_df, target_sr),1): 
    # 타겟의 비율에 맞게 나누기 때문에 split 할 때 target 데이터 제공해야 함     

    # x_trian, x_val 데이터셋 설정 
    x_train, y_train = feature_df.iloc[train_index.tolist()], target_sr[train_index.tolist()]
    x_val, y_val = feature_df.iloc[val_index.tolist()], target_sr[val_index.tolist()]


    # 학습 진행 
    dtc_model.fit(x_train, y_train)

    # 평가 -> 분류의 경우 score() 메서드 => 정확도 반환 
    train_acc = dtc_model.score(x_train, y_train)
    val_acc = dtc_model.score(x_val, y_val)

    accuracys.append([train_acc, val_acc])
    print(f'[{idx}번째] Train 정확도 : {train_acc}  Val 정확도 : {val_acc}')

[1번째] Train 정확도 : 1.0  Val 정확도 : 0.9666666666666667
[2번째] Train 정확도 : 1.0  Val 정확도 : 0.9666666666666667
[3번째] Train 정확도 : 1.0  Val 정확도 : 0.9
[4번째] Train 정확도 : 1.0  Val 정확도 : 0.9333333333333333
[5번째] Train 정확도 : 1.0  Val 정확도 : 1.0


In [300]:
# 평균 계산 
train_mean = sum([value[0] for value in accuracys]) / skFold.n_splits
val_mean = sum([value[1] for value in accuracys]) / skFold.n_splits

print(f'Train 정확도 : {train_mean}, Val 정확도 : {val_mean:.2f}')

Train 정확도 : 1.0, Val 정확도 : 0.95


교차 검증 및 성능 평가 동시 진행 함수
- cross_val_score, cross_val_predict
- cross_validate

In [301]:
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate

# 주로 cross_validate 사용 

In [302]:
# [1] 전체 DS -> 학습용과 테스트용 DS 분리

x_train, x_test, y_train, y_test = train_test_split(feature_df, target_sr, stratify= target_sr)

In [303]:
# cross_val_predict
predict = cross_val_predict(dtc_model, x_train, y_train)

print(f'predict : {predict}')

predict : [0 0 0 2 1 0 2 2 2 0 1 0 0 1 0 2 1 0 0 0 2 1 1 0 0 0 1 2 1 1 1 1 1 0 0 1 1
 1 0 1 2 1 2 0 1 1 0 2 0 2 0 2 0 1 1 0 2 2 2 2 1 2 2 2 0 0 1 2 2 0 1 1 1 0
 1 2 1 0 0 0 2 0 1 1 1 1 0 1 2 2 1 2 1 2 1 2 0 0 2 1 0 1 1 2 2 1 0 2 2 2 1
 0]


In [304]:
predict = cross_val_predict(dtc_model, x_train, y_train,cv = 3)

print(f'predict : {predict}')

predict : [0 0 0 2 1 0 2 2 2 0 1 0 0 1 0 2 1 0 0 0 2 1 1 0 0 0 1 2 1 1 1 1 1 0 0 1 1
 1 0 1 2 1 2 0 1 1 0 2 0 2 0 2 0 1 1 0 2 2 2 2 1 2 2 2 0 0 1 1 2 0 1 1 1 0
 1 2 1 0 0 0 2 0 1 1 1 1 0 1 2 2 2 2 1 2 1 2 0 0 2 1 0 1 1 2 2 1 0 2 2 2 1
 0]


In [305]:
# cross_val_score
cross_val_score(dtc_model,x_train,y_train)

array([0.95652174, 0.91304348, 0.86363636, 0.90909091, 0.95454545])

In [306]:
# crss_validate
cross_validate(dtc_model, x_train, y_train)

# 기본 5개로 쪼개고 훈련 시간과 score 함께 출력 

{'fit_time': array([0.        , 0.01234794, 0.        , 0.        , 0.01200461]),
 'score_time': array([0.        , 0.00151014, 0.        , 0.        , 0.00144196]),
 'test_score': array([0.95652174, 0.91304348, 0.90909091, 0.90909091, 0.95454545])}

In [307]:
cross_validate(dtc_model, x_train, y_train, return_train_score=True)

{'fit_time': array([0.        , 0.00926137, 0.        , 0.        , 0.        ]),
 'score_time': array([0.        , 0.        , 0.00800037, 0.        , 0.0044682 ]),
 'test_score': array([0.95652174, 0.91304348, 0.90909091, 0.90909091, 0.95454545]),
 'train_score': array([1., 1., 1., 1., 1.])}

In [308]:
result = cross_validate(dtc_model, x_train, y_train, return_train_score=True, return_estimator=True)
result
# return_estimator=True : 사용된 모델의 인스턴스를 함께 제공 
# 학습용 데이터 제공 , 아무것도 안 줘서 기본으로 5개 쪼갬 
# cv = 10으로 제공하면 10개로 쪼개서 10개가 나옴 
# -> 많이 쪼갠다고 좋은 것 아님 => 검증 데이터가 적어짐 
# -> default : 5인 이유는 일반적으로 적합 

{'fit_time': array([0., 0., 0., 0., 0.]),
 'score_time': array([0.        , 0.01029229, 0.00700879, 0.        , 0.        ]),
 'estimator': [DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier()],
 'test_score': array([0.95652174, 0.91304348, 0.90909091, 0.90909091, 0.95454545]),
 'train_score': array([1., 1., 1., 1., 1.])}

In [309]:
result_df = pd.DataFrame(result)

In [310]:
result_df

,fit_time,score_time,estimator,test_score,train_score
0,0.0,0.000000,DecisionTreeClassifier(),0.956522,1.0
1,0.0,0.010292,DecisionTreeClassifier(),0.913043,1.0
2,0.0,0.007009,DecisionTreeClassifier(),0.909091,1.0
3,0.0,0.000000,DecisionTreeClassifier(),0.909091,1.0
4,0.0,0.000000,DecisionTreeClassifier(),0.954545,1.0


In [311]:
result_df = result_df.loc[:, ['test_score','train_score']]
result_df

# 둘의 차이가 가장 적은 모델이 가장 좋은 모델 
# 1.0 1.0으로 나오면 과대적합 의심 

,test_score,train_score
0,0.956522,1.0
1,0.913043,1.0
2,0.909091,1.0
3,0.909091,1.0
4,0.954545,1.0


최적화된 모델 추출

In [312]:
best_model = result['estimator'][1]
best_model

DecisionTreeClassifier()

테스트 데이터로 확인

In [313]:
best_model.predict(x_test)

array([0, 0, 1, 2, 1, 2, 2, 0, 1, 2, 0, 2, 1, 1, 0, 0, 2, 0, 2, 1, 0, 0,
       1, 1, 0, 0, 2, 1, 1, 2, 0, 2, 1, 2, 2, 0, 1, 1])

In [314]:
best_model.score(x_test,y_test)

0.9736842105263158